In [ ]:
import pandas as pd

df=pd.read_csv("/content/after prepocessing IKN.csv")
df.head()

,Unnamed: 0,created_at,full_text,Label,tokenize text,stop removal text,stemmed_text
0,0,Wed Jan 31 21:21:49 +0000 2024,baru tau aneh kan terus masih mau bangun ikn m...,0.0,"['baru', 'tau', 'aneh', 'kan', 'terus', 'masih...","['baru', 'tau', 'aneh', 'kan', 'terus', 'mau',...",baru tau aneh kan terus mau bangun ikn makan g...
1,1,Tue Mar 26 15:43:32 +0000 2024,intinya tetap salah pemerintah sih kenapa akse...,0.0,"['intinya', 'tetap', 'salah', 'pemerintah', 's...","['intinya', 'tetap', 'salah', 'pemerintah', 's...",inti tetap salah perintah sih akses jalan jele...
2,2,Wed Jan 31 04:27:47 +0000 2024,maksudnya berkelanjutan fod estate gagal hilir...,0.0,"['maksudnya', 'berkelanjutan', 'fod', 'estate'...","['maksudnya', 'berkelanjutan', 'fod', 'estate'...",maksud lanjut fod estate gagal hilir nikel ruw...
3,3,Thu Mar 28 02:29:49 +0000 2024,lah kan itu salah satu dosanya apa mau disebut...,0.0,"['lah', 'kan', 'itu', 'salah', 'satu', 'dosany...","['lah', 'kan', 'salah', 'satu', 'dosanya', 'ap...",lah kan salah satu dosa apa mau sebut semua fo...
4,4,Wed Mar 27 08:40:25 +0000 2024,ribet karena ada ikn bayangkan ikn gagal,0.0,"['ribet', 'karena', 'ada', 'ikn', 'bayangkan',...","['ribet', 'ada', 'ikn', 'bayangkan', 'ikn', 'g...",ribet ada ikn bayang ikn gagal


In [ ]:
df.info()

In [ ]:

udalabel = df[df['Label'].notna()]
# udalabel.dropna(inplace=True)
belumlabel = df[df['Label'].isna()]
# belumlabel.dropna(inplace=True)

In [ ]:
# Bagian Word2Vec
from gensim.models import Word2Vec
import numpy as np
import pickle

# Fungsi untuk mendapatkan rata-rata vektor kata
def average_word_vectors(words, model, vector_size):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Pisahkan data menjadi fitur (full_text) dan target (label)
X = udalabel['stemmed_text']
y = udalabel['Label']

# Tokenisasi teks (diasumsikan setiap teks sudah dipisahkan dengan spasi)
X_tokenized = [text.split() for text in X]

# Latih model Word2Vec secara eksplisit
vector_size = 100  # Dimensi vektor kata
word2vec_model = Word2Vec(sentences=X_tokenized, vector_size=vector_size, window=5, min_count=1, workers=4, seed=45,sg=0)

# Simpan model Word2Vec ke file
model_path = "word2vec_model.pkl"
with open(model_path, "wb") as file:
    pickle.dump(word2vec_model, file)
    print("Model Word2Vec berhasil dilatih dan disimpan ke file.")

Model Word2Vec berhasil dilatih dan disimpan ke file.


In [ ]:
words = list(word2vec_model.wv.key_to_index.keys())
vectors = word2vec_model.wv[words]

# Membuat DataFrame
df_vectors = pd.DataFrame(vectors, index=words, columns=[f'V{i+1}' for i in range(vector_size)])

df_vectors.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100
ikn,-0.330547,-0.130553,0.046332,-0.079536,0.163343,0.191007,-0.164119,0.048519,0.180566,-0.316230,...,0.194795,-0.171149,-0.097274,0.101265,0.152206,-0.195681,-0.139803,0.229381,0.112286,0.274111
kota,-0.183719,-0.074590,0.032544,-0.044239,0.094912,0.098359,-0.084655,0.032826,0.099933,-0.176331,...,0.097504,-0.090124,-0.060516,0.044528,0.070965,-0.102189,-0.073122,0.120805,0.049860,0.155038
ibu,-0.162669,-0.050205,0.012445,-0.041383,0.063172,0.091585,-0.068351,0.022961,0.084720,-0.150783,...,0.084847,-0.075568,-0.048299,0.036761,0.054035,-0.080248,-0.075604,0.099397,0.054822,0.113974
bangun,-0.218298,-0.079082,0.033977,-0.040490,0.107038,0.118642,-0.106497,0.032455,0.120694,-0.188770,...,0.123394,-0.090320,-0.057474,0.048779,0.082121,-0.134770,-0.090310,0.138807,0.069432,0.167255
jadi,-0.178690,-0.074913,0.018432,-0.035759,0.090141,0.092826,-0.084964,0.027537,0.085876,-0.172860,...,0.106084,-0.086778,-0.046959,0.053388,0.084302,-0.109635,-0.075989,0.128411,0.065848,0.136978


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
X_word2vec = [average_word_vectors(text.split(), word2vec_model, vector_size=100) for text in X]
# Normalisasi data untuk Multinomial Naive Bayes
scaler = StandardScaler()
X_word2vec_normalized = scaler.fit_transform(X_word2vec)

# Bagi data menjadi training dan testing
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_word2vec_normalized, y, test_size=0.2, random_state=45)



In [ ]:
word2vec_df = pd.DataFrame(X_word2vec)
word2vec_df['stemmed_text'] = X.values
word2vec_df['Label'] = y.values

word2vec_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,stemmed_text,Label
0,-0.087504,-0.035085,0.009368,-0.020297,0.041755,0.047812,-0.045790,0.009597,0.044638,-0.080689,...,-0.024786,0.024389,0.040031,-0.051927,-0.037697,0.058326,0.029792,0.069694,baru tau aneh kan terus mau bangun ikn makan g...,0.0
1,-0.066092,-0.027756,0.006616,-0.013726,0.030352,0.037679,-0.033804,0.007111,0.034849,-0.058701,...,-0.016548,0.018520,0.030531,-0.036924,-0.028400,0.043189,0.021839,0.056105,inti tetap salah perintah sih akses jalan jele...,0.0
2,-0.077505,-0.029639,0.009057,-0.019739,0.037509,0.043741,-0.038761,0.011383,0.039602,-0.072094,...,-0.021780,0.021263,0.032856,-0.048217,-0.035455,0.050892,0.024877,0.061689,maksud lanjut fod estate gagal hilir nikel ruw...,0.0
3,-0.053685,-0.021147,0.005997,-0.014420,0.026248,0.030286,-0.026443,0.007783,0.028219,-0.051229,...,-0.015601,0.015463,0.023589,-0.030816,-0.024509,0.035969,0.018688,0.042280,lah kan salah satu dosa apa mau sebut semua fo...,0.0
4,-0.154018,-0.057902,0.022910,-0.034311,0.070382,0.089304,-0.076163,0.020133,0.082446,-0.146444,...,-0.042625,0.042017,0.070544,-0.088784,-0.064798,0.104062,0.053463,0.126863,ribet ada ikn bayang ikn gagal,0.0
5,-0.042573,-0.016530,0.008889,-0.009482,0.020316,0.027087,-0.019187,0.006659,0.020557,-0.041999,...,-0.011260,0.014949,0.019777,-0.022920,-0.021243,0.029083,0.014562,0.037401,punya urat malu jangan lupa bulan jelek jelek ...,0.0
6,-0.054285,-0.020637,0.006882,-0.013360,0.028264,0.029934,-0.026277,0.008381,0.029944,-0.052711,...,-0.014416,0.013325,0.023855,-0.033160,-0.027162,0.035923,0.017162,0.042441,prestasi presiden amp calon presiden hebat hal...,0.0
7,-0.043728,-0.016516,0.005755,-0.010692,0.021175,0.023799,-0.021159,0.004382,0.024387,-0.040602,...,-0.012271,0.011977,0.019938,-0.026982,-0.019693,0.028201,0.014280,0.036218,pokok politik identitas uang muka gtgtgtgtgtgt...,0.0
8,-0.081382,-0.031362,0.010788,-0.019023,0.039870,0.047181,-0.039572,0.009986,0.044286,-0.077695,...,-0.024233,0.023615,0.034918,-0.047269,-0.037264,0.055357,0.027010,0.064531,tahun merdeka butuh rakyat didik sehat mana pe...,0.0
9,-0.114697,-0.041654,0.018843,-0.023308,0.056243,0.062084,-0.056633,0.017033,0.062632,-0.110474,...,-0.037210,0.033078,0.047163,-0.069704,-0.048014,0.079205,0.031268,0.093128,ikn penuh dusta yakin mangkrak,0.0


In [ ]:
# Latih model Multinomial Naive Bayes
from sklearn.naive_bayes import GaussianNB

# Ganti MultinomialNB dengan GaussianNB
classifier = GaussianNB()
classifier.fit(X_train1, y_train1)

# Prediksi dan evaluasi model
y_pred1 = classifier.predict(X_test1)
akurasi1 = accuracy_score(y_test1, y_pred1)
print(f'Akurasi prediksi: {akurasi1 * 100:.2f}%')
print("Laporan klasifikasi:")
print(classification_report(y_test1, y_pred1))



Akurasi prediksi: 47.46%
Laporan klasifikasi:
              precision    recall  f1-score   support

         0.0       0.43      0.78      0.56        37
         1.0       0.63      0.34      0.44        35
         2.0       0.47      0.33      0.38        46

    accuracy                           0.47       118
   macro avg       0.51      0.48      0.46       118
weighted avg       0.51      0.47      0.46       118



In [ ]:
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Latih model Random Forest
classifier = RandomForestClassifier(max_depth = 30, n_estimators=10, max_features='sqrt', min_samples_leaf=2, min_samples_split=2,random_state=45)
classifier.fit(X_train1, y_train1)

# Prediksi dan evaluasi model
y_pred1 = classifier.predict(X_test1)

# Evaluasi akurasi dan laporan klasifikasi
akurasi1 = accuracy_score(y_test1, y_pred1)
print(f'Akurasi prediksi: {akurasi1 * 100:.2f}%')
print("Laporan klasifikasi:")
print(classification_report(y_test1, y_pred1))


Akurasi prediksi: 60.17%
Laporan klasifikasi:
              precision    recall  f1-score   support

         0.0       0.55      0.59      0.57        37
         1.0       0.59      0.69      0.63        35
         2.0       0.68      0.54      0.60        46

    accuracy                           0.60       118
   macro avg       0.60      0.61      0.60       118
weighted avg       0.61      0.60      0.60       118



In [ ]:
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,
KNeighborsClassifier)
from sklearn.pipeline import Pipeline


# Latih model K-Nearest Neighbors
nca = NeighborhoodComponentsAnalysis(random_state=45)
knn = KNeighborsClassifier(n_neighbors=3)



nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train1, y_train1)


y_pred = nca_pipe.predict(X_test1)
akurasi = accuracy_score(y_test1, y_pred)
print(f'Akurasi prediksi: {akurasi * 100:.2f}%')
print("Laporan klasifikasi:")
print(classification_report(y_test1, y_pred))


Akurasi prediksi: 65.25%
Laporan klasifikasi:
              precision    recall  f1-score   support

         0.0       0.61      0.73      0.67        37
         1.0       0.61      0.66      0.63        35
         2.0       0.75      0.59      0.66        46

    accuracy                           0.65       118
   macro avg       0.66      0.66      0.65       118
weighted avg       0.66      0.65      0.65       118



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
# Latih model Support Vector Machine
classifier = SVC(kernel='linear', random_state=45)
classifier.fit(X_train1, y_train1)

# Prediksi dan evaluasi model
y_pred1 = classifier.predict(X_test1)

# Evaluasi akurasi dan laporan klasifikasi
akurasi1 = accuracy_score(y_test1, y_pred1)
print(f'Akurasi prediksi: {akurasi1 * 100:.2f}%')
print("Laporan klasifikasi:")
print(classification_report(y_test1, y_pred1))


Akurasi prediksi: 66.10%
Laporan klasifikasi:
              precision    recall  f1-score   support

         0.0       0.66      0.78      0.72        37
         1.0       0.55      0.66      0.60        35
         2.0       0.81      0.57      0.67        46

    accuracy                           0.66       118
   macro avg       0.67      0.67      0.66       118
weighted avg       0.69      0.66      0.66       118



In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Latih model Decision Tree
classifier = DecisionTreeClassifier(random_state=45)
classifier.fit(X_train1, y_train1)

# Prediksi dan evaluasi model
y_pred1 = classifier.predict(X_test1)

# Evaluasi akurasi dan laporan klasifikasi
akurasi1 = accuracy_score(y_test1, y_pred1)
print(f'Akurasi prediksi: {akurasi1 * 100:.2f}%')
print("Laporan klasifikasi:")
print(classification_report(y_test1, y_pred1))

Akurasi prediksi: 55.93%
Laporan klasifikasi:
              precision    recall  f1-score   support

         0.0       0.55      0.46      0.50        37
         1.0       0.49      0.57      0.53        35
         2.0       0.63      0.63      0.63        46

    accuracy                           0.56       118
   macro avg       0.56      0.55      0.55       118
weighted avg       0.56      0.56      0.56       118

